In [ ]:
%pip install ultralytics
%matplotlib inline

In [ ]:
from ultralytics import YOLO
import numpy as np
import os
import random
from matplotlib import pyplot as plt
import cv2

In [ ]:
# Download the pre-trained nano model.
model = YOLO('yolov8n.pt')
# Start training
model.train(data='data.yaml', epochs=100, imgsz=512)

In [ ]:
# Continue training.
model = YOLO('runs/detect/train12/weights/last.pt')
model.train(resume=True)

In [ ]:
# Use trained model.
model = YOLO('runs/detect/train12/weights/best.pt')

In [ ]:
# Validate on test set.

metrics = model.val(
    data='data.yaml',
    split='test'
)
print(metrics)

In [ ]:
#print(metrics)

# for k, v in metrics.results_dict.items():
#     print(k, v)

# precision = 0.8220600246650015
# recall = 0.8052526793479584
# f1 = 2 / ((1 / precision) + (1 / recall))
# print(f1)

print(metrics.box.maps)

In [ ]:
def iou(expected, got):
    """
    ### Description

    Calculates the IoU for the given 2 boxes.

    ### Parameters

    - `expected`, `got`.

    Order doesn't matter.
    An array (or np.array) of 4 elements: [left, up, right, bottom] normalized to [0, 1].

    ### Returns

    Intersection over union (float).
    """

    xA = max(expected[0], got[0])
    yA = max(expected[1], got[1])
    xB = min(expected[2], got[2])
    yB = min(expected[3], got[3])

    area_expected = (expected[2] - expected[0]) * (expected[3] - expected[1])
    area_got = (got[2] - got[0]) * (got[3] - got[1])

    area_intersection = (xB - xA) * (yB - yA)
    area_union = area_expected + area_got - area_intersection

    iou = area_intersection / area_union
    return iou

In [ ]:
def get_input_filenames_without_extension(path):
    """
    Returns a list of the filenames for all the inputs, without the extension.
    In other words, if the directory has the following files:
    ```
        agri_0_0.jpeg
        agri_0_0.txt
        agri_0_1.jpeg
        agri_0_0.txt
    ```
    The output will be:
    ```
        ['agri_0_0', 'agri_0_1']
    ```
    """

    f = []
    for (_, _, filenames) in os.walk(path):
        f.extend([f.split(".")[0] for f in filenames])
        break
    return list(set(f))

In [ ]:

TEST_PATH = "datasets/mydataset/test/"
fnames = get_input_filenames_without_extension(TEST_PATH)
print(fnames)

In [ ]:
def bad_iou_selector(truth, got):
    truth2 = [t for t in truth]
    got2 = [g for g in got]

    iou_li = []

    for t in truth2:
        # Get best from 'g'.
        best = None
        best_iou = None

        for g in got2:
            v = iou(t['xywh'], g['xywh'])
            if best_iou is None or v > best_iou:
                best_iou = v
                best = g
        if best is not None:
            iou_li.append(best_iou)
            try:
                got2.remove(best)
            except:
                print(got2, best)
        else:
            # No more elements in 'got'.
            break

    return iou_li


In [ ]:
ious_all = []
for fname in fnames:
    print(fname)
    fname_img = f"{fname}.jpeg"
    fname_truth = f"{fname}.txt"

    truth = []

    with open(TEST_PATH + '/' + fname_truth) as f:
        lines = f.readlines()
        for l in lines:
            items = l.split()
            truth.append({
                'class': int(items[0]),
                'xywh': np.array([float(x) for x in items[1:]])
            })

    got = []

    res = model(TEST_PATH +  fname_img)
    for r in res:
        for b in r.boxes:
            # print(int(b.cls[0]))
            # print(float(b.conf[0]))
            # print(np.array(b.xywh / 255)[0])

            got.append({
                'class': int(b.cls[0]),
                'xywh': np.array([float(f) for f in (b.xywh / 512)[0]]),
                'confidence': float(b.conf[0])
            })

    ious = bad_iou_selector(truth, got)
    if len(ious) == 0:
        ious_all.append(0)
    else:
        ious_all.append(max(ious))

print("All:")
print(ious_all)


In [ ]:
metrics = model.val(
    data='data.yaml',
    split='test'
)
print(metrics)

In [ ]:
PR_curve = metrics.curves_results[0]
x = PR_curve[0]
y = PR_curve[1][0]
average_precision = np.trapz(y, x)

print(average_precision)

In [ ]:
from matplotlib import patches
import time

def eval_img(fname):
    print(fname)
    fname_img = f"{fname}.jpeg"
    fname_truth = f"{fname}.txt"

    got = []
    res = model(TEST_PATH +  fname_img)
    for r in res:
        for b in r.boxes:
            # print(int(b.cls[0]))
            # print(float(b.conf[0]))
            # print(np.array(b.xywh / 255)[0])

            got.append({
                'class': int(b.cls[0]),
                'xywh': np.array([float(f) for f in (b.xywh / 512)[0]]),
                'confidence': float(b.conf[0])
            })
    print(got)

    img = cv2.imread(TEST_PATH +  fname_img)

    fig, ax = plt.subplots(1) 
    ax.imshow(img) 

    for g in got:
        x, y, w, h = g['xywh'] * 512
        x -= w/2
        y -= h/2
        print(x, y, w, h)
        rect = patches.Rectangle((x, y), w, h, linewidth=4,  edgecolor='y' if g['class'] == 0 else 'r', facecolor="none") 
        ax.add_patch(rect) 
    plt.show() 


In [ ]:
random.seed(time.time())
fname = random.choice(fnames)
eval_img(fname)